# Playlist Recommendation System

This notebook can be used to recommend songs when given a playlist of one or more songs. 

In [2]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
df = pd.read_json('pre_pivot.json')

In [6]:
df.head()

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
0,brendan.ta,4uIDigk79DeZEYV6Z5Yf4s,What Went Down,[6FQqZYVfTNQ1pCqfkwVFEa],[Foals],2016-03-26T00:48:16Z,3pTPXB3vT93AOTSVozP54o,53L6A3I9vf7rgEZnMzx54E,Mountain At My Gates,brendan.ta,1,Foals,Foals|||||Mountain At My Gates
1,brendan.ta,4sFhah3DYcJlYeT47q3rhM,In The Silence (Deluxe Version),[7xUZ4069zcyBM4Bn10NQ1c],[Ásgeir],2016-03-26T00:49:53Z,3pTPXB3vT93AOTSVozP54o,6VNo09sojPBi5mdckQkLbX,King and Cross,brendan.ta,1,Ásgeir,Ásgeir|||||King and Cross
10,brendan.ta,5cOhR878H8hC3UsxYq5Xyv,All My Friends (feat. Tinashe & Chance the Rap...,"[2FwJwEswyIUAljqgjNSHgP, 0NIIxcxNHmOoyBx03SfTC...","[Snakehips, Tinashe, Chance the Rapper]",2016-04-29T05:34:47Z,3pTPXB3vT93AOTSVozP54o,6TaqooOXAEcijL6G1AWS2K,All My Friends (feat. Tinashe & Chance the Rap...,brendan.ta,1,Snakehips___Tinashe___Chance the Rapper,Snakehips___Tinashe___Chance the Rapper|||||Al...
100,brendan.ta,07hs3meNvGu5Fp46pnLQm7,Hungry Ghost,[2N2EFVDEbp2JB8ulEUVIxp],[Violent Soho],2017-02-16T03:09:55Z,3pTPXB3vT93AOTSVozP54o,14NnWDVM8nCssfHAqWZp1T,Covered In Chrome,brendan.ta,1,Violent Soho,Violent Soho|||||Covered In Chrome
1000,brendan.ta,4PKH86wn7Gw4iel2WD564k,Do It Like You,[0aA1GTrIMutjIh4GlPPUVN],[Crooked Colours],2019-02-04T05:22:30Z,3LF6iEL6uCE9rSdLF4m1ga,5h1Gi50T9QxVfzvIF8vciC,Do It Like You,brendan.ta,1,Crooked Colours,Crooked Colours|||||Do It Like You


In [3]:
df = pd.read_json('track_data_9_18_19.json')

In [5]:
df.head()

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
0,technozem,0d0ONE5rak6Q91XjwKcJvN,Vurstep,"[7ugvHO0W3IoAWzOgKrHxqr, 0K1lHu1BP65Z1DErnljxUw]","[Appleblim, Forest Drive West]",2018-12-04T21:27:55Z,5tW6vGqn4Z2oPxpgQSncKD,5huzNc7H2kv2qNq7kqscHY,Vurstep - Forest Drive West Remix,technozem,1,Appleblim___Forest Drive West,Appleblim___Forest Drive West|||||Vurstep - Fo...
1,technozem,2qz8u01gOb8Lb7KaTR90DQ,Marble,"[7asRTH6SKIMKZZ59Iw2eA5, 2iWOFT9U8InefnarwZUmv0]","[Gnork, Douala]",2018-12-12T21:33:25Z,5tW6vGqn4Z2oPxpgQSncKD,7Hhfn4AkePvX0jW4jOJPQ3,Space Jam (feat. Douala),technozem,1,Gnork___Douala,Gnork___Douala|||||Space Jam (feat. Douala)
10,technozem,4JooAi5hHhUuJNrbiseE8X,At the Controls,[68Wb5Pcy71lLaKdIB6cBA5],[Breakage],2018-12-11T20:02:07Z,5tW6vGqn4Z2oPxpgQSncKD,5NLRRESuSXLQTPgdqAECCq,Rudeboy Stuff,technozem,1,Breakage,Breakage|||||Rudeboy Stuff
100,technozem,6fXNAGf1ihG0B0Sck2Mo1l,Metropolis,[0Ij7th9uWcDVYNAIOn5W22],[Kornél Kovács],2018-12-29T17:28:10Z,5tW6vGqn4Z2oPxpgQSncKD,0ajH7MyTiRhjUbZVYVgRPc,Panda,technozem,1,Kornél Kovács,Kornél Kovács|||||Panda
1000,technozem,3YQASaeJPm3OxUSCP6Qfo9,GROEF - Des avonds in klein maneschijn,[7DksXfhuJLdqtyHnoKCJLI],[GROEF],2011-04-15T16:06:11Z,4AehzXKZqJ5VrqPHpsmN6E,6sihMoVzSFhHtr2p556f5Z,Jan mijne man/Andro/Andro GROEF,technozem,1,GROEF,GROEF|||||Jan mijne man/Andro/Andro GROEF


In [6]:
df.shape

(1022972, 13)

In [7]:
df.dropna().shape

(1018598, 13)

In [8]:
df.shape

(1022972, 13)

In [9]:
df.dtypes

added_by_id         object
album_id            object
album_name          object
artist_ids          object
artist_names        object
date_added          object
playlist_id         object
track_id            object
track_name          object
user_id             object
rating               int64
artists_join        object
artist_and_track    object
dtype: object

In [31]:
df=df.dropna()
df=df[df.artist_and_track!='']
df = df[df.duplicated(subset='artist_and_track', keep=False)]
df.reset_index(inplace = True)
df.drop(columns = 'index', inplace = True)

In [10]:
df.shape

(1022972, 13)

In [11]:
df.artist_and_track.nunique()

437768

In [14]:
df.shape

(1022972, 13)

In [12]:
df.user_id.nunique() # these are the unique users that added playlists to either r/spotify or r/spotifyplaylists

419

In [13]:
df.added_by_id.nunique() # these are the unique users that created a spotify playlist in the database

1582

In [14]:
df.playlist_id.nunique()

8339

In [20]:
df.head()

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
0,brendan.ta,4uIDigk79DeZEYV6Z5Yf4s,What Went Down,[6FQqZYVfTNQ1pCqfkwVFEa],[Foals],2016-03-26T00:48:16Z,3pTPXB3vT93AOTSVozP54o,53L6A3I9vf7rgEZnMzx54E,Mountain At My Gates,brendan.ta,1,Foals,Foals|||||Mountain At My Gates
1,brendan.ta,4sFhah3DYcJlYeT47q3rhM,In The Silence (Deluxe Version),[7xUZ4069zcyBM4Bn10NQ1c],[Ásgeir],2016-03-26T00:49:53Z,3pTPXB3vT93AOTSVozP54o,6VNo09sojPBi5mdckQkLbX,King and Cross,brendan.ta,1,Ásgeir,Ásgeir|||||King and Cross
10,brendan.ta,5cOhR878H8hC3UsxYq5Xyv,All My Friends (feat. Tinashe & Chance the Rap...,"[2FwJwEswyIUAljqgjNSHgP, 0NIIxcxNHmOoyBx03SfTC...","[Snakehips, Tinashe, Chance the Rapper]",2016-04-29T05:34:47Z,3pTPXB3vT93AOTSVozP54o,6TaqooOXAEcijL6G1AWS2K,All My Friends (feat. Tinashe & Chance the Rap...,brendan.ta,1,Snakehips___Tinashe___Chance the Rapper,Snakehips___Tinashe___Chance the Rapper|||||Al...
100,brendan.ta,07hs3meNvGu5Fp46pnLQm7,Hungry Ghost,[2N2EFVDEbp2JB8ulEUVIxp],[Violent Soho],2017-02-16T03:09:55Z,3pTPXB3vT93AOTSVozP54o,14NnWDVM8nCssfHAqWZp1T,Covered In Chrome,brendan.ta,1,Violent Soho,Violent Soho|||||Covered In Chrome
1000,brendan.ta,4PKH86wn7Gw4iel2WD564k,Do It Like You,[0aA1GTrIMutjIh4GlPPUVN],[Crooked Colours],2019-02-04T05:22:30Z,3LF6iEL6uCE9rSdLF4m1ga,5h1Gi50T9QxVfzvIF8vciC,Do It Like You,brendan.ta,1,Crooked Colours,Crooked Colours|||||Do It Like You


| Users | Playlists   | Unique Songs   | Total Songs|
|------|------|------|------|
|   918  | 6049| 116157| 515096|

| Avg Playlists/User | Avg Songs/Playlist   |
|-----|------|
|   6.6  | 85.2|

In [15]:
def search_by_artist_and_track(df, artist, track):
    '''
    This function searches the database by artist and track. All tracks that contain substrings with an
    exact match of the artist and track strings are returned. Only the first match is printed.
    '''
    print(df[df.artist_and_track.str.contains(artist, regex=False, case=False) & df.artist_and_track.str.contains(track, regex=False, case=False)].artist_and_track.iloc[0])
    return df[df.artist_and_track.str.contains(artist, regex=False, case=False) & df.artist_and_track.str.contains(track, regex=False, case=False)]



In [16]:
def make_playlist(user_id, playlist_id, df, list_of_searches):
    '''
    Makes a playlist from a list of searches using the first songs returned by each search. Returns a dataframe
    representing the new playlist.
    '''
    new_df = pd.DataFrame()
    for artist,track in list_of_searches:
        curr = search_by_artist_and_track(df, artist, track).iloc[0]
        new_df = new_df.append(pd.DataFrame([[user_id, curr.album_id,curr.album_name,curr.artist_ids,
                   curr.artist_names,'NA',playlist_id,curr.track_id,
                   curr.track_name,user_id,1,curr.artists_join,curr.artist_and_track]], columns=df.columns))
    return new_df

In [17]:
def create_sparse(df):
    '''
    Creates a sparse matrix of the songs and playlists.
    '''
    playlist_id_c = CategoricalDtype(sorted(df.playlist_id.unique()), ordered=True)
    artist_and_track_c = CategoricalDtype(sorted(df.artist_and_track.unique()), ordered=True)

    row = df.playlist_id.astype(playlist_id_c).cat.codes
    col = df.artist_and_track.astype(artist_and_track_c).cat.codes
    sparse_matrix = csr_matrix((df['rating'], (row, col)), \
                               shape=(playlist_id_c.categories.size, artist_and_track_c.categories.size))
    return sparse_matrix, playlist_id_c

In [18]:
def create_sim_dict(playlist_id, df):
    '''
    Creates similarity dictionary from sparse matrix.
    '''
    sparse_matrix, playlist_id_c = create_sparse(df)
    similarities_playlists = cosine_similarity(sparse_matrix)
    playlists = np.array(playlist_id_c.categories)
    playlists_indices = list(playlists)
    sim_zip = zip(playlists_indices, similarities_playlists[playlists_indices.index(playlist_id)])
    sim_dict = {p:sim for p,sim in sim_zip}
    return sim_dict

In [19]:
def create_rec_df(playlist_id, df):
    '''
    Create recommendations with sum of similarities. This should be used over the mean of similarities if you
    want song popularity to influence song recommendations. Higher popularity means the song is more likely to
    be recommended. Returns recommendations, which are the songs listed in order of decreasing similarity sums.
    '''
    df_rec = df[df.playlist_id != playlist_id]
    sim_dict = create_sim_dict(playlist_id, df)
    df_sim = pd.DataFrame(zip(sim_dict.keys(), sim_dict.values()))
    df_sim.columns = ['playlist_id','sim']
    df_rec = df.merge(df_sim,how='left', on='playlist_id')
    grouped = df_rec.groupby(by = 'artist_and_track').sum()
    recommendations = grouped.sort_values(by = 'sim', ascending = False)
    return recommendations

In [20]:
def recommend_for_playlist(user_id, playlist_id, df, list_of_searches):
    '''
    Temporarily adds an input playlist into a dataframe in order to return similarities. Returns recommendations.
    Currently, it's likely that the input playlist songs are included in the recommendations. This is intentional
    in order to set up future work with the spotify api but may be changed in a later version.
    '''
    my_playlist = make_playlist(user_id, playlist_id, df, list_of_searches)
    df = df.append(my_playlist)
    recommendations = create_rec_df(playlist_id, df)
    return recommendations

In [21]:
def simplify(messy_rec, number_tracks):
    '''
    Creates a clean dataframe from recommendations, which is the return from the recommend_for_playlist functions.
    '''
    combined = []
    for row in messy_rec.head(number_tracks).itertuples():
        combined.append(row.Index.split('|||||'))
    clean_df = pd.DataFrame(np.array(combined))
    clean_df.columns = ['artist','track']
    return clean_df

In [22]:
def create_rec_df_mean(playlist_id, df):
    '''
    Create recommendations with mean of similarities. This should be used over the sum of similarities if you
    DO NOT want song popularity to influence song recommendations. This is a good option if you find you're
    getting the same recommendations often. Returns recommendations, which are the songs listed in order of
    decreasing similarity means.
    '''
    df_rec = df[df.playlist_id != playlist_id]
    sim_dict = create_sim_dict(playlist_id, df)
    df_sim = pd.DataFrame(zip(sim_dict.keys(), sim_dict.values()))
    df_sim.columns = ['playlist_id','sim']
    df_rec = df.merge(df_sim,how='left', on='playlist_id')
    grouped = df_rec.groupby(by = 'artist_and_track').mean()
    recommendations = grouped.sort_values(by = 'sim', ascending = False)
    return recommendations

In [23]:
def recommend_for_playlist_mean(user_id, playlist_id, df, list_of_searches):
    '''
    Temporarily adds an input playlist into a dataframe in order to return similarities. Returns recommendations.
    Currently, it's likely that the input playlist songs are included in the recommendations. This is intentional
    in order to set up future work with the spotify api but may be changed in a later version.
    '''
    my_playlist = make_playlist(user_id, playlist_id, df, list_of_searches)
    df = df.append(my_playlist)
    recommendations = create_rec_df_mean(playlist_id, df)
    return recommendations

In [26]:
simplify(recommend_for_playlist('justin', 'justins jams', df, [('croce','i got a name'),
                                                               ('steely dan','do it again'),
                                                              ('spinners','rubberband')]), 30)

Jim Croce|||||I Got A Name
Steely Dan|||||Do It Again
The Spinners|||||The Rubberband Man


,artist,track
0,The Spinners,The Rubberband Man
1,Steely Dan,Do It Again
2,Jim Croce,I Got A Name
3,Electric Light Orchestra,Mr. Blue Sky
4,George Harrison,My Sweet Lord
5,Yusuf / Cat Stevens,Father And Son
6,Looking Glass,Brandy (You're a Fine Girl)
7,Rupert Holmes,Escape (The Pina Colada Song)
8,The Doobie Brothers,Listen to the Music
9,Blue Swede___Björn Skifs,Hooked on a Feeling


In [28]:
simplify(recommend_for_playlist_mean('justin', 'justinsjams', df, [('croce','i got a name'),
                                                     ('steely dan','do it again')]), 20)

Jim Croce|||||I Got A Name
Steely Dan|||||Do It Again


,artist,track
0,Katie Hanley___Godspell Ensemble,By My Side
1,Carpenters,Hurting Each Other
2,Olivia Newton-John,Sam
3,The Five Stairsteps,Ooh Child
4,The Delfonics,Funny Feeling
5,Jim Croce,I Got A Name
6,The Feelies,Raised Eyebrows
7,Gloria Gaynor,Never Can Say Goodbye (Re-Recorded)
8,Misfits___Glenn Danzig,Teenagers From Mars
9,The Dillinger Escape Plan,Symptom of Terminal Illness


In [29]:
simplify(recommend_for_playlist_mean('Dan', 'Dans jams', df, [('led','stairway'),
                                                             ('paul','band on the run')]), 30)

Led Zeppelin|||||Stairway to Heaven - 1990 Remaster
Paul McCartney___Wings|||||Band On The Run - Remastered 2010


,artist,track
0,Led Zeppelin,When the Levee Breaks - Alternate UK Mix in Pr...
1,Dee Snider___Diego Boneta,I Wanna Rock
2,Jani Lane___Nuno Bettencourt___Gary F. Cherone...,More Than Words / Heaven
3,Lou Gramm___Mick Jones___Julianne Hough___Dieg...,Waiting For A Girl Like You
4,Aaron Cabott Jones,Wish Upon a Starfish
5,Mr Krax,A Tight Little Band
6,The Beatles,While My Guitar Gently Weeps - 2018 Mix
7,The Beatles,Help! - Live / Remastered
8,The Beatles,Can't Buy Me Love - Live / Remastered
9,The Beatles,Shout - Anthology 1 Version
